<a href="https://colab.research.google.com/github/t1seo/AIFFEL_Hackerthon-1/blob/main/taewon/air_hpg_info.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/MyDrive/data/Restaurant_Visitor_Forecasting

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import warnings
warnings.filterwarnings("ignore")

In [ ]:
air_reserve = pd.read_csv('air_reserve.csv')
hpg_reserve = pd.read_csv('hpg_reserve.csv')
air_store_info = pd.read_csv('air_store_info.csv')
hpg_store_info = pd.read_csv('hpg_store_info.csv')
store_id_relation = pd.read_csv('store_id_relation.csv')
air_visit_data = pd.read_csv('air_visit_data.csv')
sample_submission = pd.read_csv('sample_submission.csv')
date_info = pd.read_csv('date_info.csv')

## Air Store Info

In [ ]:
air_store_info.head()

In [ ]:
print("Air Store Info Row: ", len(air_store_info))

- 총 829개의 row가 있다.

### air_store_info 업종별 식당 개수 파악

In [ ]:
air_genre = air_store_info[['air_genre_name']]
air_genre.head()

In [ ]:
air_genre.value_counts()

- `air_store_info`에 있는 업종별 개수를 파악해보았더니 `Izakaya`가 가장 많고 그 다음으로 `Cafe/Sweets', 'Dining bar` 순으로 많다.

In [ ]:
air_genre = air_store_info["air_genre_name"].value_counts().reset_index()
air_genre = air_genre.rename(columns={"index":"air_genre", "air_genre_name":"count"})
air_genre

In [ ]:
plt.figure(figsize=(15, 10))
sns.set(font_scale=2) 
sns.barplot(y = air_genre["air_genre"], x = air_genre["count"])

- 막대 그래프로 시각화 한 결과 `air_store_info`에서 `Izakaya`업종이 가장 많은 것을 확인할 수 있다.

### air_store_info 지역별 식당 개수 파악

In [ ]:
air_area = air_store_info["air_area_name"].value_counts().reset_index()
air_area = air_area.rename(columns={"index":"air_area", "air_area_name":"count"})
air_area

In [ ]:
# top 15 지역
air_area_15 = air_area.loc[:15, :]
air_area_15

- `air_store_info`의 식당들은 `Fukuoka`와 `Tokyo`에 많은 위치해 있는 것을 확인할 수 있다. 

In [ ]:
plt.figure(figsize=(15, 10))
sns.barplot(y = air_area_15["air_area"], x = air_area_15["count"])

- 막대 그래프로 시각화 한 결과도 마찬가지로 후쿠오카와 도쿄에 식당들이 많이 위치해 있는 것을 확인할 수 있다.

### Air Info 지도

Air Info에 있는 식당들이 어느 지역에 많이 분포되어 있는지 지도로 확인해본다.

In [ ]:
import folium
from folium import plugins

folium.__version__

In [ ]:
air_store_info.head()

In [ ]:
location = air_store_info.groupby(['air_store_id', 'air_genre_name'])['latitude', 'longitude'].mean().reset_index()
lacationlist = location[['latitude', 'longitude']]
locationlist = lacationlist.values.tolist()

map = folium.Map(location=[39, 139], 
                        tiles = "Stamen Toner",# width=1000, height=500,
                        zoom_start = 5)
marker_cluster=plugins.MarkerCluster().add_to(map)
for point in range(0, len(location)):
    folium.Marker(locationlist[point], popup=location['air_genre_name'][point], 
    icon=folium.Icon(color='white', icon_color='red', 
                     #icon='fa fa-info-circle',
                     icon='fa fa-circle-o-notch fa-spin',
                     angle=0, 
                     prefix='fa')).add_to(marker_cluster)
map

## HPG Store Info

In [ ]:
hpg_store_info.head()

In [ ]:
print("HOG Store Info Row: ", len(hpg_store_info))

- `hpg`에는 `air`(829)보다 더 많은 row들이 있다.

### HPG Store 업종별 식당 개수 파악

In [ ]:
hpg_genre = hpg_store_info[['hpg_genre_name']]
hpg_genre.head()

In [ ]:
hpg_genre.value_counts()

- `hpg_store_info`의 `genre`를 확인해보니 `Japanse_style`이 가장 많다. 이를 시각화 해보자.

In [ ]:
hpg_genre = hpg_genre["hpg_genre_name"].value_counts().reset_index()
hpg_genre = hpg_genre.rename(columns={"index":"hpg_genre", "hpg_genre_name":"count"})
hpg_genre

- `air_store_info`보다 더 다양한 업종이 있는 것을 볼 수 있다.
- `Japananese Style`의 식당이 가장 많은 것을 확인해 볼 수 있다.

In [ ]:
plt.figure(figsize=(15, 10))
plt.rc('ytick', labelsize=14)
sns.barplot(y = hpg_genre["hpg_genre"], x = hpg_genre["count"])

- `Japanese style`의 식당이 가장 많고 `International cuisine`과 `Creation`이 뒤를 잇는다.

### hpg_store 지역별 식당 개수 파악

In [ ]:
hpg_area = hpg_store_info["hpg_area_name"].value_counts().reset_index()
hpg_area = hpg_area.rename(columns={"index":"hpg_area", "hpg_area_name":"count"})
hpg_area

In [ ]:
hpg_area_15 = hpg_area.loc[:15, :]
hpg_area_15

- `air`에는 후쿠오카의 식당들이 가장 많았었는데 `hpg`에는 도쿄 신주쿠의 식당이 가장 많이 있다.

In [ ]:
plt.figure(figsize=(15, 10))
sns.barplot(y = hpg_area_15["hpg_area"], x = hpg_area_15["count"])

- `hpg`에는 도쿄 지역의 식당이 가장 많다.

### HPG Store Info 지도

In [ ]:
import folium
from folium import plugins

folium.__version__

In [ ]:
hpg_store_info.head()

In [ ]:
location = hpg_store_info.groupby(['hpg_store_id', 'hpg_genre_name'])['latitude', 'longitude'].mean().reset_index()
lacationlist = location[['latitude', 'longitude']]
locationlist = lacationlist.values.tolist()

map = folium.Map(location=[39, 139], 
                        tiles = "Stamen Toner",
                        zoom_start = 5)
marker_cluster=plugins.MarkerCluster().add_to(map)
for point in range(0, len(location)):
    folium.Marker(locationlist[point], popup=location['hpg_genre_name'][point], 
    icon=folium.Icon(color='white', icon_color='red', 
                     icon='fa fa-circle-o-notch fa-spin',
                     angle=0, 
                     prefix='fa')).add_to(marker_cluster)
map